# Parser for Orca Scans

In [16]:
import cclib
import pybel
import glob
import os
import pandas as pd

In [23]:
from chemreps.utils.molecule import Molecule
from math import sqrt
import re

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


def length(molecule, atomi, atomj):
    """
    Returns the length between two atoms
    Parameters
    -----------
    molecule : object
        molecule object
    atomi, atomj : int
        atoms
    Returns
    --------
    rij : float
        length between the two
    """
    x = molecule.xyz[atomi][0] - molecule.xyz[atomj][0]
    y = molecule.xyz[atomi][1] - molecule.xyz[atomj][1]
    z = molecule.xyz[atomi][2] - molecule.xyz[atomj][2]
    rij = sqrt((x ** 2) + (y ** 2) + (z ** 2))
    return rij

molfiles = ['HF', 'CO', 'H2', 'H2O', 'HCCH', 'H2CCH2', 'HCN', 'NH3', 'N2', 'CH4', 'CH3OH', 'C6H6-C', 'C6H6-H']
stretches = ['FH', 'OC', 'HH', 'OH', 'CC', 'CC', 'NC', 'NH', 'NN', 'CH', 'OC', 'CC', 'CH']

In [28]:
elems = {1:'H', 6:'C', 7:'N', 8:'O', 9:'F'}

energies = []
for file in sorted(glob.iglob('*.out')):
    molecule = file.split('.')[0]
    data = cclib.io.ccread(file)
    for i in range(len(data.atomcoords)):
        if i == 0:
            pass
        else:
            # get geometery and energy 
            geom = data.atomcoords[i]
            energy = data.scfenergies[i]
            natom = data.natom
            # write xyz file
            with open('{}/xyz/{}.xyz'.format(molecule, i), 'w') as fxyz:
                print(natom, file=fxyz)
                print('Energy: \t{}'.format(data.scfenergies[i]), file=fxyz)
                for j in range(natom):
                    sym = elems[data.atomnos[j]]
                    x = geom[j][0]
                    y = geom[j][1]
                    z = geom[j][2]
                    print('{} \t\t{} \t\t{} \t\t{}'.format(sym, x, y, z), file=fxyz)
            
            # open with pybel in order to make sdf
            mol = next(pybel.readfile('xyz', '{}/xyz/{}.xyz'.format(molecule, i)))
            # rewrite xyz to make cleaner
            output = pybel.Outputfile('xyz', '{}/xyz/{}.xyz'.format(molecule, i), overwrite=True)
            output.write(mol)
            output.close()
            # write sdf file
            output = pybel.Outputfile('sdf', '{}/sdf/{}.sdf'.format(molecule, i), overwrite=True)
            output.write(mol)
            output.close()
            
            # grab bond length and stretch
            btype = stretches[molfiles.index(molecule)]
            current_molecule = Molecule('{}/sdf/{}.sdf'.format(molecule, i))
            doubcount = 0
            for k in range(current_molecule.n_atom):
                for l in range(k, current_molecule.n_atom):
                    atomi = current_molecule.sym[k]
                    atomj = current_molecule.sym[l]
                    zi = current_molecule.at_num[k]
                    zj = current_molecule.at_num[l]
                    if k == l:
                        mii = 0.5 * zi ** 2.4
                    else:
                        if zj > zi:
                            # swap ordering
                            atomi, atomj = atomj, atomi
                        bond = "{}{}".format(atomi, atomj)
                        if bond == btype:
                            doubcount += 1
                            if doubcount == 1:
                                # rij = sqrt((xi - xj)^2 + (yi - yj)^2 + (zi - zj)^2)
                                rij = length(current_molecule, k, l)
                                
            # store in dict to append to list
            d = {}
            d.update({'name': molecule})
            d.update({'point': i})
            d.update({'bond': btype})
            d.update({'length': rij})
            d.update({'energy': energy})
            energies.append(d)
            

In [29]:
df = pd.DataFrame(energies)
df

,name,point,bond,length,energy
0,C6H6-C,1,CC,0.9846,-6259.129633
1,C6H6-C,2,CC,1.0846,-6290.919938
2,C6H6-C,3,CC,1.1846,-6307.793560
3,C6H6-C,4,CC,1.2846,-6315.569917
4,C6H6-C,5,CC,1.3846,-6317.796290
...,...,...,...,...,...
580,NH3,41,NH,4.6126,-1512.751874
581,NH3,42,NH,4.7126,-1512.665388
582,NH3,43,NH,4.8126,-1512.585553
583,NH3,44,NH,4.9126,-1512.511668


In [30]:
df.to_csv('../data/dft-data.csv', index=False)